# **PHẦN: Lấy dữ liệu từ 3 website bán vàng " SJC, DOJI, PNJ"**

# **LẤY DỮ LIỆU TỪ SJC**

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import time

# Import SQLAlchemy & GoldPrice model
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import GoldPrice,DATABASE_URL  # ✅ import từ file models.py

# Cấu hình Selenium để chạy nền
options = Options()
options.headless = True

# Kết nối PostgreSQL
DB_USER = 'laravel'
DB_PASSWORD = 'npg_VdWDJ3MZ2kjz'
DB_HOST = 'ep-billowing-star-a1a7fumd.aws-ap-southeast-1.pg.laravel.cloud'
DB_PORT = 5432
DB_NAME = 'main'

# Khởi tạo WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Truy cập URL
url = "https://sjc.com.vn/gia-vang-online"
driver.get(url)
time.sleep(3)

# Tìm bảng giá vàng
gold_table = driver.find_element(By.CLASS_NAME, 'sjc-table-show-price-online')
rows = gold_table.find_elements(By.TAG_NAME, 'tr')

# Kết nối database
engine = create_engine(
    DATABASE_URL, echo=True
)
Session = sessionmaker(bind=engine)
session = Session()

# Duyệt các hàng dữ liệu
for row in rows[1:]:  # Bỏ header
    columns = row.find_elements(By.TAG_NAME, 'td')
    if len(columns) >= 3:
        gold_type = columns[0].text.strip()
        buy_price = columns[1].text.strip()
        sell_price = columns[2].text.strip()

        gold_entry = GoldPrice(
            source="SJC",
            gold_type=gold_type,
            buy_price=buy_price,
            sell_price=sell_price,
            date=datetime.today().date(),
            time=datetime.today().time(),
            scraped_at=datetime.utcnow()
        )

        session.add(gold_entry)
        print(f"✅ Đã lưu: {gold_type} - Mua: {buy_price} - Bán: {sell_price}")

# Lưu thay đổi & đóng kết nối
session.commit()
session.close()
driver.quit()
print("✅ Crawl và lưu dữ liệu hoàn tất.")


C:\Users\Asus\AppData\Local\Temp\ipykernel_6808\2954573341.py:59: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  scraped_at=datetime.utcnow()


✅ Đã lưu: VÀNG SJC 1L, 10L, 1KG - Mua: 119,000,000 - Bán: 121,000,000
✅ Đã lưu: VÀNG SJC 5 CHỈ - Mua: 119,000,000 - Bán: 121,020,000
✅ Đã lưu: VÀNG SJC 0.5 CHỈ, 1 CHỈ, 2 CHỈ - Mua: 119,000,000 - Bán: 121,030,000
✅ Đã lưu: VÀNG NHẪN SJC 99,99% 1 CHỈ, 2 CHỈ, 5 CHỈ - Mua: 113,500,000 - Bán: 116,000,000
✅ Đã lưu: VÀNG NHẪN SJC 99,99% 0.5 CHỈ, 0.3 CHỈ - Mua: 113,500,000 - Bán: 116,100,000
✅ Đã lưu: NỮ TRANG 99,99% - Mua: 113,500,000 - Bán: 115,400,000
✅ Đã lưu: NỮ TRANG 99% - Mua: 109,757,000 - Bán: 114,257,000
✅ Đã lưu: NỮ TRANG 75% - Mua: 80,209,000 - Bán: 86,709,000
✅ Đã lưu: NỮ TRANG 68% - Mua: 72,130,000 - Bán: 78,630,000
✅ Đã lưu: NỮ TRANG 61% - Mua: 64,051,000 - Bán: 70,551,000
✅ Đã lưu: NỮ TRANG 58,3% - Mua: 60,935,000 - Bán: 67,435,000
✅ Đã lưu: NỮ TRANG 41,7% - Mua: 41,777,000 - Bán: 48,277,000
2025-05-12 11:17:29,188 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-05-12 11:17:29,189 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-12 11:17:29,366 INFO sqlalchemy.

# **LẤY DỮ LIỆU TỪ DOJI**

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from datetime import datetime

# Import SQLAlchemy & GoldPrice model
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import GoldPrice,DATABASE_URL# ✅ import từ file models.py

# Cấu hình để chạy Selenium ở chế độ headless (ẩn trình duyệt)
options = Options()
options.headless = True

# Khởi tạo driver Chrome
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Truy cập trang giá vàng của Doji
url = "http://update.giavang.doji.vn/system/doji_92411/92411"
driver.get(url)

# Đợi trang và JavaScript render xong
time.sleep(5)  # Tăng lên 5 giây để chắc chắn

# Lấy ngày và giờ
try:
    date = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'day'))
    ).text.strip()
    
    # Đổi định dạng ngày tháng từ '8-5-2025' thành '2025-05-08'
    date_obj = datetime.strptime(date, '%d-%m-%Y')
    date = date_obj.strftime('%Y-%m-%d')  # '2025-05-08'
except:
    date = "N/A"

try:
    time_text = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//em[contains(text(), "giờ")]/following-sibling::span'))
    ).text.strip()
except:
    time_text = "N/A"

print(f"Ngày: {date} - Giờ: {time_text}")

# Kết nối database
engine = create_engine(
    DATABASE_URL, echo=False
)
Session = sessionmaker(bind=engine)
session = Session()

# Lặp qua từng loại vàng
while True:
    try:
        # Lấy lại các phần tử giá vàng
        rows = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[ng-repeat="bg in bgs"]'))
        )
    
        for row in rows:
            try:
                gold_type = row.find_element(By.CSS_SELECTOR, '.table-cell:first-child').text.strip()
                buy_price_raw = row.find_element(By.CSS_SELECTOR, '.table-cell.buy').text.strip()
                sell_price_raw = row.find_element(By.CSS_SELECTOR, '.table-cell.send').text.strip()

                # Chuyển từ "nghìn VNĐ/chỉ" => VNĐ/lượng
                buy_price = float(buy_price_raw.replace(",", "")) * 10000
                sell_price = float(sell_price_raw.replace(",", "")) * 10000

                # Format lại thành chuỗi có dấu phẩy
                buy_price_str = f"{int(buy_price):,}"
                sell_price_str = f"{int(sell_price):,}"

                print(f"{date} {time_text} - Loại vàng: {gold_type} - Mua: {buy_price_str} VNĐ/lượng - Bán: {sell_price_str} VNĐ/lượng")

                gold_price_entry = GoldPrice(
                    source="DOJI",
                    gold_type=gold_type,
                    buy_price=buy_price_str,
                    sell_price=sell_price_str,
                    date=date,
                    time=time_text,
                    scraped_at=datetime.utcnow()
                )

                session.add(gold_price_entry)
                session.commit()

            except Exception as e:
                print(f"Error: {e}")
                continue

        
    except Exception as e:
        print(f"Error during scraping: {e}")
        break

# Đóng trình duyệt và session
driver.quit()
session.close()


Ngày: 2025-05-12 - Giờ: 11:23
2025-05-12 11:23 - Loại vàng: MIẾNG SJC (LOẠI 1L) - BÁN LẺ - Mua: 119,000,000 VNĐ/lượng - Bán: 121,000,000 VNĐ/lượng


C:\Users\Asus\AppData\Local\Temp\ipykernel_6808\1607530242.py:91: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  scraped_at=datetime.utcnow()


Error: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=136.0.7103.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0071FC53+61635]
	GetHandleVerifier [0x0071FC94+61700]
	(No symbol) [0x005405D3]
	(No symbol) [0x00546DD0]
	(No symbol) [0x005490CA]
	(No symbol) [0x00549147]
	(No symbol) [0x00588234]
	(No symbol) [0x00588D3B]
	(No symbol) [0x0057E111]
	(No symbol) [0x005AD2E4]
	(No symbol) [0x0057E034]
	(No symbol) [0x005AD514]
	(No symbol) [0x005CE61B]
	(No symbol) [0x005AD096]
	(No symbol) [0x0057C840]
	(No symbol) [0x0057D6A4]
	GetHandleVerifier [0x009A4573+2701795]
	GetHandleVerifier [0x0099FCF6+2683238]
	GetHandleVerifier [0x009BAA3E+2793134]
	GetHandleVerifier [0x00736915+155013]
	GetHandleVerifier [0x0073CFFD+181357]
	GetHandleVerifier [0x007274A8+92440]
	GetHandleVerifier [0x00727650+92

KeyboardInterrupt: 

# **LẤY DỮ LIỆU TỪ PNJ**

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Import SQLAlchemy & GoldPrice model
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import GoldPrice  # ✅ import từ file models.py

# Thiết lập chế độ headless (ẩn trình duyệt)
options = Options()
options.headless = True

# Khởi tạo WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Truy cập trang giá vàng của PNJ
url = "https://www.pnj.com.vn/site/gia-vang"
driver.get(url)

# Chờ trang tải JavaScript xong
time.sleep(5)

# ======= LẤY NGÀY VÀ GIỜ ========
def convert_date_format(date_str):
    try:
        return datetime.strptime(date_str, '%d/%m/%Y').date()
    except ValueError:
        return None  # Nếu không thể chuyển đổi, trả về None

try:
    # Tìm phần tử chứa text "Cập nhật ngày:"
    update_text = driver.find_element(By.CSS_SELECTOR, 'p.text-sm.text-gray-400.mt-1').text
    # Trích xuất ngày và giờ từ chuỗi bằng regex
    match = re.search(r'(\d{2}/\d{2}/\d{4}) (\d{2}:\d{2})', update_text)
    if match:
        date = match.group(1)
        time_updated = match.group(2)
        date = convert_date_format(date)  # Chuyển đổi định dạng ngày
    else:
        date = time_updated = "N/A"
except:
    date = time_updated = "N/A"

print(f"Ngày cập nhật: {date} - Giờ cập nhật: {time_updated}")

# ======= LẤY GIÁ VÀNG ============
try:
    # Mỗi dòng nằm trong thẻ <tr>, trong đó các cột là loại vàng, giá mua, giá bán
    rows = driver.find_elements(By.CSS_SELECTOR, 'table tbody tr')

    # Kết nối database
    engine = create_engine(
        DATABASE_URL, echo=False
    )
    Session = sessionmaker(bind=engine)
    session = Session()

    for row in rows:
        columns = row.find_elements(By.TAG_NAME, 'td')
        if len(columns) >= 3:
            gold_type = columns[0].text.strip()
            buy_price_raw = columns[1].text.strip()
            sell_price_raw = columns[2].text.strip()

            # Quy đổi từ nghìn đồng/chỉ -> VNĐ/lượng
            try:
                buy_price = float(buy_price_raw.replace(",", "")) * 10000
                sell_price = float(sell_price_raw.replace(",", "")) * 10000
            except ValueError:
                print(f"Không thể chuyển đổi giá: {buy_price_raw}, {sell_price_raw}")
                continue

            # Format lại thành chuỗi có dấu phẩy phân cách
            buy_price_str = f"{int(buy_price):,}"
            sell_price_str = f"{int(sell_price):,}"

            print(f"{date} {time_updated} - Loại vàng: {gold_type} - Mua: {buy_price_str} VNĐ/lượng - Bán: {sell_price_str} VNĐ/lượng")

            # Lưu dữ liệu vào database
            gold_price_entry = GoldPrice(
                source="PNJ",
                gold_type=gold_type,
                buy_price=buy_price_str,
                sell_price=sell_price_str,
                date=date,
                time=time_updated,
                scraped_at=datetime.utcnow()
            )

            session.add(gold_price_entry)
            session.commit()


except Exception as e:
    print("Lỗi:", e)

# Đóng trình duyệt và session
driver.quit()
session.close()


Ngày cập nhật: 2025-05-12 - Giờ cập nhật: 08:54
2025-05-12 08:54 - Loại vàng: Vàng miếng SJC 999.9 - Mua: 119,000,000 VNĐ/lượng - Bán: 121,000,000 VNĐ/lượng


C:\Users\Asus\AppData\Local\Temp\ipykernel_6808\1701315023.py:95: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  scraped_at=datetime.utcnow()


2025-05-12 08:54 - Loại vàng: Nhẫn Trơn PNJ 999.9 - Mua: 113,500,000 VNĐ/lượng - Bán: 116,000,000 VNĐ/lượng
2025-05-12 08:54 - Loại vàng: Vàng Kim Bảo 999.9 - Mua: 113,500,000 VNĐ/lượng - Bán: 116,000,000 VNĐ/lượng
2025-05-12 08:54 - Loại vàng: Vàng Phúc Lộc Tài 999.9 - Mua: 113,500,000 VNĐ/lượng - Bán: 116,000,000 VNĐ/lượng
2025-05-12 08:54 - Loại vàng: Vàng nữ trang 999.9 - Mua: 113,000,000 VNĐ/lượng - Bán: 115,500,000 VNĐ/lượng
2025-05-12 08:54 - Loại vàng: Vàng nữ trang 999 - Mua: 112,890,000 VNĐ/lượng - Bán: 115,390,000 VNĐ/lượng
2025-05-12 08:54 - Loại vàng: Vàng nữ trang 9920 - Mua: 112,180,000 VNĐ/lượng - Bán: 114,680,000 VNĐ/lượng
2025-05-12 08:54 - Loại vàng: Vàng nữ trang 99 - Mua: 111,950,000 VNĐ/lượng - Bán: 114,450,000 VNĐ/lượng
2025-05-12 08:54 - Loại vàng: Vàng 750 (18K) - Mua: 79,280,000 VNĐ/lượng - Bán: 86,780,000 VNĐ/lượng
2025-05-12 08:54 - Loại vàng: Vàng 585 (14K) - Mua: 60,220,000 VNĐ/lượng - Bán: 67,720,000 VNĐ/lượng
2025-05-12 08:54 - Loại vàng: Vàng 416 (10K) 